Automatic DeepFake Creation (Tortoise voice cloning + wav2lip)

Note: A video is required. If an audio is also provided, the voice will be cloned from the audio.

# **The video should have a face looking at all times to the camera.**

## **Video (and audio) file should be in Google drive in a folder named 'deepfake'. No other files should exist there**

wav2lip code taken from https://github.com/snehitvaddi/Deepfake-using-Wave2Lip

In [1]:
#@title Upload video.mp4 (video to overlay voice) & voice.mp3 (voice to clone) files - Should be mp3 and mp4, having any name
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/deepfake

/content/gdrive/MyDrive/deepfake


In [3]:
base_path='/content/gdrive/MyDrive/deepfake' #Specify path of video/audio

In [4]:
#@title Install TTS, pydub to create folders with audio chunks, and moviepy to modify duration of audio/video
!pip install -q pydub==0.25.1 TTS==0.22.0 moviepy==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 93.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 100.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB

In [5]:
#@title English text that we want to read with the cloned voice - This will be inserted in the video too
#Text prompt should be separated with '|' every one to two sentences (every ~20secs it takes to read it).
text_to_read="""Joining two modalities results in a surprising increase in generalization! |
                What would happen if we combined them all? """

In [6]:
#@title Rename audio and video files to be used below
#Prompt: the user selects and uploads to google colab one audio and one video file. Rename the audio file to 'input_voice.mp3' and the video to 'input_video.mp4'
import os

# Loop over files in the directory
for file in os.listdir(os.getcwd()):

      filename = os.path.join(base_path, file)

      if filename.endswith('.mp3'):
          new_filename = 'input_voice.mp3'
          os.rename(filename, new_filename)
      if filename.endswith('.mp4'):
          new_filename = 'video_full.mp4'
          os.rename(filename, new_filename)

#If only video is provided:
from moviepy.editor import VideoFileClip

def extract_audio(input_video, output_audio):
    video = VideoFileClip(input_video)
    audio = video.audio
    audio.write_audiofile(output_audio)

# Provide the input video file path and desired output audio file path
input_video = 'video_full.mp4'
output_audio = 'input_voice.mp3'

#Decide if voice will be cloned from video or audio
mp3_check=0
for file in os.listdir(os.getcwd()):
      file_path =  os.path.join(base_path, file)
      if '.mp3' in file_path:
        mp3_check=1

if mp3_check==0:
  print("Voice will be cloned from video")
  extract_audio(input_video, output_audio)

  if event.key is 'enter':



Voice will be cloned from video
MoviePy - Writing audio in input_voice.mp3


MoviePy - Done.


In [7]:
#@title Create folder with 10 secs chunks of audio to be used as input in Tortoise
from pydub import AudioSegment

def split_audio_to_clips(audio_file, output_dir, clip_length=10000, sample_rate=22050):
    # Load the audio file
    audio = AudioSegment.from_mp3(audio_file)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Calculate the total number of clips
    num_clips = len(audio) // clip_length

    # Split the audio into clips and save them as WAV files
    for i in range(num_clips):
        start_time = i * clip_length
        end_time = start_time + clip_length
        clip = audio[start_time:end_time]

        # Set the sample width to 2 bytes for floating-point format
        clip = clip.set_sample_width(2)

        # Set the sample rate to 22050 Hz
        clip = clip.set_frame_rate(sample_rate)

        # Save the clip as a WAV file
        clip.export(os.path.join(output_dir, f"{i+1}.wav"), format="wav")

    print(f"{num_clips} clips saved in '{output_dir}'.")

if __name__ == "__main__":

    # Replace 'input_audio.mp3' with the name of your MP3 file
    input_audio_file = base_path+'/input_voice.mp3'

    # Replace 'voices' with the desired subdirectory name
    subdirectory_name = base_path+'/voice'

    split_audio_to_clips(input_audio_file, subdirectory_name)

15 clips saved in '/content/gdrive/MyDrive/deepfake/voice'.


In [8]:
#@title Download and run TTS tortoise model
from TTS.api import TTS
tts = TTS("tts_models/en/multi-dataset/tortoise-v2")

 > Downloading model to /root/.local/share/tts/tts_models--en--multi-dataset--tortoise-v2


100%|█████████▉| 1.71G/1.72G [00:21<00:00, 88.0MiB/s]
100%|██████████| 1.72G/1.72G [00:22<00:00, 76.1MiB/s]

  1%|          | 8.60M/976M [00:00<00:11, 86.0MiB/s]
  2%|▏         | 17.2M/976M [00:00<00:12, 76.7MiB/s]
  3%|▎         | 24.9M/976M [00:00<00:13, 72.1MiB/s]
  3%|▎         | 32.2M/976M [00:00<00:13, 71.1MiB/s]
  4%|▍         | 40.5M/976M [00:00<00:12, 75.0MiB/s]
  5%|▍         | 48.0M/976M [00:00<00:14, 64.0MiB/s]
  6%|▌         | 54.8M/976M [00:00<00:14, 65.2MiB/s]
  6%|▋         | 62.9M/976M [00:01<00:18, 48.5MiB/s]
  7%|▋         | 71.5M/976M [00:01<00:15, 56.8MiB/s]
  8%|▊         | 78.0M/976M [00:01<00:21, 42.0MiB/s]
  9%|▊         | 84.5M/976M [00:01<00:19, 46.5MiB/s]
 10%|▉         | 94.4M/976M [00:01<00:16, 53.6MiB/s]
 11%|█         | 104M/976M [00:01<00:13, 63.1MiB/s] 
 11%|█▏        | 111M/976M [00:01<00:13, 65.6MiB/s]
 12%|█▏        | 118M/976M [00:01<00:12, 66.9MiB/s]
 13%|█▎        | 126M/976M [00:02<00:12, 68.5MiB/s]
 14%|█▍        | 136M/976M [00:02<00:10, 76.4M

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: tortoise


  checkpoint = torch.load(ar_path, map_location=torch.device("cpu"))

  self.diffusion.load_state_dict(torch.load(diff_path), strict=strict)

  self.clvp.load_state_dict(torch.load(clvp_path), strict=strict)

  torch.load(

GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model archite

In [9]:
#@title Install tortoise original repository to be used to merge cloned audio pieces together
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 2.1.3 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [10]:
from tortoise.utils.text import split_and_recombine_text
from time import time

# Process text
text = text_to_read
if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text) #If only one piece of text (<20secs), convert it to list

Found the '|' character in your text, which I will use as a cue for where to split it up. If this was notyour intent, please remove all '|' characters from the input.


In [11]:
cd ..

/content/gdrive/MyDrive/deepfake


In [1]:
#Tortoise Fastest Inference from script (~2min per 20secs of audio in Colab)
#Can also be downloaded from https://huggingface.co/jbetker/tortoise-tts-v2/tree/main
import torch
import torchaudio

from TTS.tts.configs.tortoise_config import TortoiseConfig
from TTS.tts.models.tortoise import Tortoise
import IPython
import gc

config = TortoiseConfig()
model = Tortoise.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/root/.local/share/tts/tts_models--en--multi-dataset--tortoise-v2", eval=True) #Deepspeed doesn't work
model.cuda()

all_parts = []
for j, text in enumerate(texts): #Combine individuals pieces of text by processing sound one piece at a time (around 20secs each)
  # cloning a speaker
  output_dict = model.synthesize(text, config, speaker_id="voice", voice_dirs=base_path)

  #Save result
  torchaudio.save("tortoise_v2_script_"+str(j)+".wav", torch.tensor(output_dict["wav"]).squeeze(0), 24000)
  all_parts.append(torch.tensor(output_dict["wav"]).squeeze(0))
  del output_dict

  # Perform garbage collection
  gc.collect()

torchaudio.save('combined.wav', torch.cat(all_parts, dim=-1), 24000)
IPython.display.Audio('combined.wav')

Clean memory so that we can have a bit longer text to read

In [13]:
del all_parts

In [14]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [15]:
import gc
# Perform garbage collection
gc.collect()

21

In [16]:
from google.colab import files
files.download('/content/gdrive/MyDrive/deepfake/combined.wav')
#Individual pieces can also be found in this folder

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
#@title Confirm that audio same length as video. If not, keep the smallest one and cut the other (or cut them both to 20secs). This is needed for wav2lip to work
#ChatGPT Prompt: Create python code that compares an audio.wav with a video.mp4 files and if the duration of one is bigger than the other,
# it cuts the largest one to be the same duration as the smallest. If any of them is bigger than 20secs then raise an error

#Needed to avoid errors with encoding
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from moviepy.editor import VideoFileClip, AudioFileClip
import os

def compare_audio_video_duration(audio_file, video_file):
    audio = AudioFileClip(audio_file)
    video = VideoFileClip(video_file)

    audio_duration = audio.duration
    video_duration = video.duration

    # Either the video or audio should be <20 secs. If any of these is larger than that, it will be cut to the duration of the other or to 20secs.
    # Might work for up to 30secs, but not guaranteed. If only video is provided, it will keep only the first 20 secs of it.

    # if audio_duration > 20 and video_duration > 20:
    #     video = video.subclip(0, 20)
    #     video.write_videofile('input_video.mp4')
    #     audio = audio.subclip(0, 20)
    #     audio.write_audiofile('input_audio.wav')

    if audio_duration != video_duration:
        min_duration = min(audio_duration, video_duration)
        if min_duration == audio_duration:
            video = video.subclip(0, min_duration)
            video.write_videofile('input_video.mp4')
            os.rename(audio_file,'input_audio.wav')
        else:
            audio = audio.subclip(0, min_duration)
            audio.write_audiofile('input_audio.wav')
            os.rename(video_file,'input_video.mp4')

    audio.close()
    video.close()

# Example usage
compare_audio_video_duration("combined.wav", "video_full.mp4") #input_voice.mp3

100%|██████████| 4.40k/4.40k [05:07<00:00, 14.3iB/s]

Moviepy - Building video input_video.mp4.
MoviePy - Writing audio in input_videoTEMP_MPY_wvf_snd.mp3



chunk:  85%|████████▌ | 364/427 [00:00<00:00, 1194.86it/s, now=None]
100%|██████████| 4.40k/4.40k [05:07<00:00, 14.3iB/s]

MoviePy - Done.
Moviepy - Writing video input_video.mp4




t: 100%|██████████| 484/484 [00:11<00:00, 42.34it/s, now=None]
100%|██████████| 4.40k/4.40k [05:19<00:00, 13.8iB/s]

Moviepy - Done !
Moviepy - video ready input_video.mp4




---



In [18]:
cd /content

/content


In [19]:
#@title <h1>Install Wav2Lip</h1>
#@markdown * Install dependencies
#@markdown * Download models
# !rm -rf /content/sample_data
# !mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O /content/Wav2Lip/checkpoints/wav2lip.pth
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
from IPython.display import clear_output
clear_output()
print("\nDone")


Done




---



- Below implementation needs both audio and video to be of same length. Only specific extensions work (mp4 and wav)
- Target face in the input_video.mp4, must be "detectable" in ALL videoframes (So no black or blurry frames etc)
- wav2lip does not like very long and high res clips (1080p/30seconds recommended)
- 'Wav2Lip' model gives highly accurate lip-sync compared to 'Wav2Lip + GAN' but with inferior visual quality compared to the latter

Below is needed to fix an error in loading - Not added in the beginning due to conflict in dependencies

In [20]:
!pip install librosa==0.9.1

  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.3 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires einops>=0.6.0, but you have einops 0.5.0 which is incompatible.
tts 0.22.0 requires librosa>=0.10.0, but you have librosa 0.9.1 which is incompatible.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 2.0.2 which is incompatible.
tts 0.22.0 requires transformers>=4.33.0, but you have transformers 4.19.0 which is incompatible.


In [21]:
# Clear all user-defined variables except built-in ones
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

# Perform garbage collection to free up memory
import gc
gc.collect()

7768

In [25]:
!pip install -U numpy==2.0.0 pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires einops>=0.6.0, but you have einops 0.5.0 which is incompatible.
tts 0.22.0 requires librosa>=0.10.0, but you have librosa 0.9.1 which is incompatible.
tts 0.22.0 requires numpy==1.22.0; p

In [26]:
#@title Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/gdrive/MyDrive/deepfake/input_video.mp4' --audio '/content/gdrive/MyDrive/deepfake/input_audio.wav' --resize_factor 2

#Use --resize_factor 2 otherwise OOM error. Use resize_factor to reduce the video resolution, as there is a chance you might get better results for lower resolution videos.
# This might be related with the model which might have been trained on low resolution faces.

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 484
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 1548)
Length of mel chunks: 481
  0% 0/4 [00:00<?, ?it/s]/content/Wav2Lip/face_detection/detection/sfd/sfd_detector.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer b

In [2]:
#@title Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [28]:
#@title Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4') #Only after the last cell is executed this will start
files.download('/content/gdrive/MyDrive/deepfake/input_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# #@title Delete old uploaded samples & result files, so you can start over again.
# # %rm /content/sample_data/*
# %rm /content/Wav2Lip/results/*
# from IPython.display import clear_output
# clear_output()
# print("\nDone! now press X")

# **Variations to try**


In [ ]:
# #@title Create Wav2Lip video using wav2lip.pth
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"  --resize_factor 2

In [ ]:
#@title Use more padding to include the chin region (you can manually edit pads dimensions viewing and changing the code)
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0 --resize_factor 2

In [ ]:
# #@title Play result video -  50% scaling
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML(f"""
# <video width="50%" height="50%" controls>
#       <source src="{data_url}" type="video/mp4">
# </video>""")

In [ ]:
#@title Download Result.mp4 to your computer
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4')